In [1]:
# Libraries
library(R.matlab)
library(ggplot2)
library(lattice)
library(Boruta)
library(mlbench)
library(caret)
library(randomForest)
library(data.table)
library(magrittr)
library(dplyr)

R.matlab v3.6.2 (2018-09-26) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# loading csv data
df_ad8_ <- fread(input="./dataset/AD8主觀認知障礙[0-1,>=2][mean].csv")
df_moca_ <- fread(input = "./dataset/MOCA客觀認知測驗分數[<=23,>=24][mean].csv")
# print(colnames(df_outcome))
# print(colnames(df_value))

In [3]:
# remove subj patient smaple name
df_ad8_ <- df_ad8_[ ,setdiff(colnames(df_ad8_), 'Subj'), with=FALSE]
df_moca_ <- df_moca_[ ,setdiff(colnames(df_moca_), 'Subj'), with=FALSE]
head(df_ad8_,3)
head(df_moca_,3)

1_unassigned,10_unassigned,100_DefaultMode,101_DefaultMode,102_DefaultMode,103_DefaultMode,104_DefaultMode,105_DefaultMode,106_DefaultMode,107_DefaultMode,⋯,91_DefaultMode,92_DefaultMode,93_DefaultMode,94_DefaultMode,95_DefaultMode,96_DefaultMode,97_DefaultMode,98_DefaultMode,99_DefaultMode,outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0.7061689,0.8575445,0.7052558,0.7982326,0.7138551,0.7378071,0.9653893,1.022310,2.035718,1.111226,⋯,0.8121306,0.8154143,0.7535726,0.7508472,1.2553424,0.4121088,0.7589343,1.2509834,1.310278,1
1.0849213,0.8849982,1.0307140,1.1849732,1.0972043,1.0459110,1.4902368,1.160965,2.124221,1.343091,⋯,0.8265397,1.3062833,1.4046950,1.1461399,1.9450089,1.7712055,1.0357548,1.3217860,1.118118,1
0.7289415,0.3793027,1.0309113,2.1049465,0.9905913,0.8511426,1.6731609,1.573165,3.194865,1.180309,⋯,0.8923878,1.3893740,0.8931532,1.3763585,0.8761261,1.2406956,1.0028500,0.9780304,1.197015,5


1_unassigned,10_unassigned,100_DefaultMode,101_DefaultMode,102_DefaultMode,103_DefaultMode,104_DefaultMode,105_DefaultMode,106_DefaultMode,107_DefaultMode,⋯,91_DefaultMode,92_DefaultMode,93_DefaultMode,94_DefaultMode,95_DefaultMode,96_DefaultMode,97_DefaultMode,98_DefaultMode,99_DefaultMode,outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0.3717417,0.2522757,0.8975669,1.422525,0.7762359,0.6675160,1.275439,1.0858376,1.763560,1.0175207,⋯,0.8928204,1.833623,1.5664630,0.9330516,1.168554,0.5361413,0.7684926,1.216418,1.232424,29
0.7856422,0.7837878,0.9562682,1.082560,0.7710374,0.8022433,1.031848,0.9062507,1.509226,0.9429646,⋯,0.8928404,1.019845,0.9143425,1.0407412,1.741321,2.3785811,0.7637207,1.096368,1.074213,26
1.0849213,0.8849982,1.0307140,1.184973,1.0972043,1.0459110,1.490237,1.1609652,2.124221,1.3430908,⋯,0.8265397,1.306283,1.4046950,1.1461399,1.945009,1.7712055,1.0357548,1.321786,1.118118,31


In [4]:
set.seed(123)
# boruta_AD8 <- Boruta(outcome ~ ., data = df_ad8_, doTrace = 0, maxRuns = 500)
# boruta_MOCA <- Boruta(outcome ~ ., data = df_moca_, doTrace = 0, maxRuns = 500)

In [5]:
# Feature Selection
set.seed(123)
boruta_AD8 <- Boruta(outcome ~ ., data = df_ad8_, doTrace = 0, maxRuns = 500)
boruta_MOCA <- Boruta(outcome ~ ., data = df_moca_, doTrace = 0, maxRuns = 500)

In [6]:
AD8_shadow <- attStats(boruta_AD8)
MOCA_shadow <- attStats(boruta_MOCA)
MOCA_shadow <- MOCA_shadow[order(MOCA_shadow$'normHits', decreasing = TRUE),]
AD8_shadow <- AD8_shadow[order(AD8_shadow$'normHits', decreasing = TRUE),]

In [7]:
head(MOCA_shadow, 10)

,meanImp,medianImp,minImp,maxImp,normHits,decision
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
`255_SomatomotorLateral`,11.79879957,12.6938872,1.8822461,15.7868138,0.93103448,Confirmed
`60_CinguloOpercular`,6.68259323,7.3854498,-2.0090982,11.4300647,0.81896552,Confirmed
`1_unassigned`,4.42985141,4.4565216,-0.6528898,8.9412304,0.68965517,Confirmed
`44_SomatomotorDorsal`,0.78339497,0.7334220,-1.9770764,4.2442487,0.12931034,Rejected
`253_SomatomotorDorsal`,0.53314028,0.7302286,-1.6144744,2.9657606,0.03448276,Rejected
`155_Visual`,-0.25360959,-0.3686951,-1.3745151,2.1443678,0.00862069,Rejected
`180_FrontoParietal`,-0.30311669,-0.6824830,-2.9104800,2.0220433,0.00862069,Rejected
`190_FrontoParietal`,-0.06876686,-0.4928795,-1.4092344,2.1443923,0.00862069,Rejected
`10_unassigned`,-0.76240938,-0.8887395,-1.9039057,0.5861817,0.00000000,Rejected


In [8]:
head(AD8_shadow, 10)

,meanImp,medianImp,minImp,maxImp,normHits,decision
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
`288_FrontoParietal`,7.45530790,7.55676554,2.43526146,10.077133,0.971943888,Confirmed
`211_VentralAttention`,8.10534529,8.24609009,1.00784215,10.871136,0.955911824,Confirmed
`159_Visual`,5.03145752,5.12696190,-0.96553789,8.705427,0.849699399,Confirmed
`248_Salience`,4.86058474,4.94118700,-0.05296698,7.811495,0.827655311,Confirmed
`277_DefaultMode`,4.39920837,4.52213378,-1.14778427,7.087937,0.789579158,Confirmed
`279_DefaultMode`,3.92542672,3.98386120,0.42528716,7.069565,0.751503006,Confirmed
`65_CinguloOpercular`,3.82466419,3.81547802,0.25221041,7.637239,0.715430862,Confirmed
`285_FrontoParietal`,2.96349552,2.94625787,-0.53788619,6.364743,0.575150301,Tentative
`127_DefaultMode`,0.17333149,0.05360774,-1.84516662,2.176586,0.002004008,Rejected


In [9]:
# print(boruta_AD8)
# plot(boruta_AD8, las=2, cex.axis=1)
# plotImpHistory(boruta_AD8)
# print(boruta_MOCA)
# plot(boruta_MOCA, las=2, cex.axis=0.7)